In [1]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['twython', 'pandas', 'psycopg2-binary', 'twint', 'nest_asyncio']
pipmain(['install'] + packages)

In [2]:
import twint
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

In [3]:
%%capture
import sys
sys.path.append('../')
from get_tweet_data import *

from aws import host as ahost, port as aport, user as auser, password as apassword, database as adatabase
import psycopg2
connection = psycopg2.connect(host = ahost, 
                              port = aport, 
                              user = auser, 
                              password = apassword, 
                              dbname = adatabase)
cursor = connection.cursor()

In [8]:
FOLLOWERS_LIMIT = 10

In [5]:
twitter_handles = ['realDonaldTrump','ewarren','JoeBiden', 'SenSanders','KamalaHarris','CoryBooker','AndrewYang']
def get_followers_list(username, limit = 200):
    c = twint.Config()
    c.Limit = limit
    c.Hide_output = True
    c.Username = username
    c.Pandas = True

    twint.run.Followers(c)
    
    f = twint.storage.panda.Follow_df
    list_of_followers = f['followers'][username]
    
    return list_of_followers

In [9]:
for user in twitter_handles:
    followers = get_followers_list(user, limit = FOLLOWERS_LIMIT)
    write_user_followers_to_db(followers, user)
    print(read_user_followers_from_db(user)) #debug line

     follower_handle
0       JuniorPalesa
1       Mark00574043
2          delan_dee
3          meekreisy
4         LegendARne
..               ...
255         CHENHunt
256  William63222826
257        curtarmy1
258      menukossery
259     Moonlightas2

[260 rows x 1 columns]
    follower_handle
0    cassie17011071
1           xzhoush
2       AveryHannig
3        RylandLori
4          travo245
5        bluesrider
6         Miketabz1
7     buckroeblazer
8        lets_hotel
9        cerescupid
10      MikeOxsbig4
11      KyleHill771
12      venturinom3
13        Queenybum
14    Laura43697878
15   Sharonewalker8
16         n_ziller
17       GeekSweat_
18      BeloKendrix
19    hellarealtalk
20  hussein61091233
21         SarenSok
22     desolates0ul
23         ThornbyD
24     imaginaimage
25   kwiatek_dorota
26        jacobolto
27    ayoubchaabene
28      MrRease1911
29   Libertycrazy97
30         maythym1
31     banks_mikado
32     patti_mccune
33  MarcantelRoland
34  davidsh33355134
35  

In [39]:
twitter_handles = ['realDonaldTrump','ewarren','JoeBiden', 'SenSanders','KamalaHarris','CoryBooker','AndrewYang']
user = twitter_handles[0]
followers = read_user_followers_from_db(user)

follower = followers['follower_handle'].tolist()[0]
follower

     follower_handle
0       JuniorPalesa
1       Mark00574043
2          delan_dee
3          meekreisy
4         LegendARne
..               ...
255         CHENHunt
256  William63222826
257        curtarmy1
258      menukossery
259     Moonlightas2

[260 rows x 1 columns]


In [74]:
def get_tweets_df(follower, limit=100):
    c = twint.Config()
    c.Limit = limit
    c.Username = follower
    c.Pandas = True
    c.Retweets = False
    twint.run.Search(c)

    Tweets_df = twint.storage.panda.Tweets_df
    if len(Tweets_df) == 0:
        return pd.DataFrame()
    df = Tweets_df[['id', 'date', 'tweet', 'hashtags']]
    df.loc[:,'date'] = [datetime.datetime.strptime(s.split(' ')[0], '%Y-%m-%d') for s in df['date'].tolist()]
    df.loc[:,'id'] = df['id'].astype(int)
    return df

In [ ]:
import datetime
df = Tweets_df[['id', 'date', 'tweet', 'hashtags']]
df.loc[:,'date'] = [datetime.datetime.strptime(s.split(' ')[0], '%Y-%m-%d') for s in df['date'].tolist()]
df.loc[:,'id'] = df['id'].astype(int)
df

In [71]:
# noun based word extraction
def word_hashtag_extraction_noun_based(tweet_dictionary):  
    text = tweet_dictionary['tweet']
    text = casual_tokenize(tweet_dictionary['tweet'])
    result = nltk.pos_tag(text)
    
    all_noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
    proper_noun_tags = ['NNP', 'NNPS']
    
    cleaned_text = {}
    for t in result:
        word = t[0]
        tag = t[1]
        
        # words
        if (tag in all_noun_tags) and isEnglish(word):
            if tag in proper_noun_tags:
                cleaned_text[word.lower()] = PROPER_NOUN_WORD_TYPE 
            else:
                cleaned_text[word.lower()] = REGULAR_NOUN_WORD_TYPE
                
    hashtags = tweet_dictionary['hashtags']
    
    return cleaned_text, set([h.lower() for h in hashtags])

In [ ]:
# iterate through rows, where each row is a tweet

# df.apply(word_hashtag_extraction_noun_based)
for i in range(len(df)):
    output = word_hashtag_extraction_noun_based(df.iloc[i,:])
    print(df['id'].iloc[i])
    
    if len(output[0]) != 0:
        
    
# df.apply(lambda x: word_hashtag_extraction_noun_based(x), axis=0)
# df.head()

In [72]:
def write_tweets_words_hashtags_to_db(follower, tweets_dictionary):
    for i in range(len(df)):
        tweet = df.iloc[i,:]
        if len(tweet) == 0:
            continue
        tweet_id = tweet['id']
        tweet_date = tweet['date']
        sql = 'INSERT INTO tweets VALUES ({},\'{}\',\'{}\');'.format(tweet_id, follower, tweet_date)

        # if writing tweet to database was sucessful, then we need to get write word count info
        if write_try(sql):
            words, tags = word_hashtag_extraction_noun_based(tweet)
            
            word_count = get_freq_map(words)
            hashtag_count = get_freq_map(tags)

            for w, c in word_count.items():
                word_type_int = words[w]
                sql2 = 'INSERT INTO words VALUES ({},\'{}\',{},{});'.format(tweet_id, w, c, word_type_int)
                write_try(sql2)

            for w, c in hashtag_count.items():
                sql2 = 'INSERT INTO hashtags VALUES ({},\'{}\',{});'.format(tweet_id, w, c)
                write_try(sql2)

In [52]:
write_tweets_words_hashtags_to_db('TrevWall', df)

In [54]:
read_all_tweets_from_db()

,tweet_id,follower_handle,tweet_date
0,0,test_follower1,1977-01-08 04:05:06
1,1,test_follower1,1999-01-08 04:05:06
2,1198375711133884416,banks_mikado,2019-11-23 23:00:08
3,1198375272934002688,banks_mikado,2019-11-23 22:58:24
4,1198372948207448065,banks_mikado,2019-11-23 22:49:10
...,...,...,...
1342,1172907465555546112,TrevWall,2019-09-14 11:18:26
1343,1172622786042482688,TrevWall,2019-09-13 16:27:13
1344,1172622347637067777,TrevWall,2019-09-13 16:25:28
1345,1172613564483416064,TrevWall,2019-09-13 15:50:34


In [75]:
for user in twitter_handles:
    followers = read_user_followers_from_db(user)
    print(followers)
    for follower in followers[:10]:
        print(follower)
        df = get_tweets_df(follower)
        write_tweets_words_hashtags_to_db(follower, df)
    break

     follower_handle
0       JuniorPalesa
1       Mark00574043
2          delan_dee
3          meekreisy
4         LegendARne
..               ...
255         CHENHunt
256  William63222826
257        curtarmy1
258      menukossery
259     Moonlightas2

[260 rows x 1 columns]
follower_handle


In [81]:
read_tweet_from_db(df['id'].iloc[0])

""


In [87]:
def read_user_tweets_from_db(follower):
    sql = ('SELECT tweet_date FROM tweets '
    'WHERE follower_handle = \'{}\''.format(follower)
    )
    return read_try(sql)

In [84]:
read_user_tweets_from_db("delan_dee")

""


In [88]:
for follower in followers:
    df = read_user_tweets_from_db(follower)
    if len(df) > 0:
        print(follower)
        print(df.head())

In [86]:
followers

,follower_handle
0,JuniorPalesa
1,Mark00574043
2,delan_dee
3,meekreisy
4,LegendARne
...,...
255,CHENHunt
256,William63222826
257,curtarmy1
258,menukossery
